In [21]:
import sys
from pathlib import Path

In [22]:
src_path = Path('../src').resolve()
sys.path.append(str(src_path))

In [ ]:
from sqlmodel import Session, select
from api.db.session import engine
from api.events.models import EventModel
from timescaledb.hyperfunctions import time_bucket, time_bucket_gapfill
from pprint import pprint

In [24]:
with Session(engine) as session:
    query = select(EventModel).order_by(EventModel.updated_at.desc()).limit(10)
    compiled_query = query.compile(compile_kwargs={"literal_binds":True})
    # print(compiled_query)
    # print()
    # print(str(query))
    results = session.exec(query).fetchall()
    pprint(results)
    # result = session.exec(query).all()
    # print(result)

[EventModel(time=datetime.datetime(2025, 9, 17, 11, 58, 4, 5772, tzinfo=datetime.timezone.utc), page='/contact', updated_at=datetime.datetime(2025, 9, 17, 11, 58, 4, 5778, tzinfo=datetime.timezone.utc), id=10000, description='My Description'),
 EventModel(time=datetime.datetime(2025, 9, 17, 11, 58, 3, 988663, tzinfo=datetime.timezone.utc), page='/pages', updated_at=datetime.datetime(2025, 9, 17, 11, 58, 3, 988669, tzinfo=datetime.timezone.utc), id=9999, description='My Description'),
 EventModel(time=datetime.datetime(2025, 9, 17, 11, 58, 3, 972258, tzinfo=datetime.timezone.utc), page='pricing', updated_at=datetime.datetime(2025, 9, 17, 11, 58, 3, 972264, tzinfo=datetime.timezone.utc), id=9998, description='My Description'),
 EventModel(time=datetime.datetime(2025, 9, 17, 11, 58, 3, 955262, tzinfo=datetime.timezone.utc), page='/about', updated_at=datetime.datetime(2025, 9, 17, 11, 58, 3, 955268, tzinfo=datetime.timezone.utc), id=9997, description='My Description'),
 EventModel(time=dat

In [66]:
from sqlalchemy import func
from datetime import datetime, timedelta, timezone

with Session(engine) as session:
    bucket = time_bucket("1 minute", EventModel.time)
    pages = ['/about', '/contact', '/pages', 'pricing']
    start = datetime.now(timezone.utc) - timedelta(hours=1)
    finish = datetime.now(timezone.utc) - timedelta(seconds=278, milliseconds=600)
    query = (
        select(
            bucket,
            EventModel.page,
            func.count()
        ).where(
            EventModel.time > start,
            EventModel.time < finish,
            EventModel.page.in_(pages)
        ).group_by(
            bucket,
            EventModel.page,
        ).order_by(
            bucket,
            EventModel.page
        )
        )
    compiled_query = query.compile(compile_kwargs={"literal_binds":True})
    results = session.exec(query).fetchall()
    print(compiled_query)
    # pprint(results)
    # result = session.exec(query).all()
    # print(result)

SELECT time_bucket('1 minute'::interval, eventmodel.time) AS time_bucket_1, eventmodel.page, count(*) AS count_1 
FROM eventmodel 
WHERE eventmodel.time > '2025-09-18 05:20:40.954374+00:00' AND eventmodel.time < '2025-09-18 06:16:02.354386+00:00' AND eventmodel.page IN ('/about', '/contact', '/pages', 'pricing') GROUP BY time_bucket('1 minute'::interval, eventmodel.time), eventmodel.page ORDER BY time_bucket('1 minute'::interval, eventmodel.time), eventmodel.page
